https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE130353

In [ ]:
from __init__ import *

In [ ]:
SETTING = kraft.json.read("setting.json")

In [ ]:
gse_id = "GSE130353"

In [ ]:
file_path_format = "{}/{}_{{}}_x_sample.tsv".format(SETTING["directory_path"], gse_id)

In [ ]:
_x_samples = kraft.geo.get_gse(
    gse_id, SETTING["directory_path"], overwrite=SETTING["overwrite"]
)

In [ ]:
binary_feature_x_sample = _x_samples[2]

binary_feature_x_sample.to_csv(file_path_format.format("binary_feature"), sep="\t")

binary_feature_x_sample

In [ ]:
file_path = kraft.internet.download(
    "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE130353&format=file",
    SETTING["directory_path"],
    file_name="{}_RAW.tar".format(gse_id),
    overwrite=SETTING["overwrite"],
)

In [ ]:
samples = []

with tarfile.open(file_path) as io:

    for tarinfo in io:

        n_transcript = pd.read_csv(
            io.extractfile(tarinfo),
            compression="gzip",
            skiprows=1,
            sep="\t",
            index_col=0,
        ).iloc[:, 6]

        n_transcript.name = tarinfo.name.split(sep="_")[1]

        samples.append(n_transcript)

gene_x_sample = pd.DataFrame(data=samples).T

gene_x_sample.index = kraft.name_biology.name_genes(gene_x_sample.index.to_numpy())

gene_x_sample.index.name = "Gene"

gene_x_sample = kraft.feature_x_sample.collapse(gene_x_sample)

gene_x_sample.to_csv(file_path_format.format("gene"), sep="\t")

gene_x_sample